**Code:** https://mer.vin/2024/02/semantic-chunking/

In [12]:
!pip install -q -U sentence-transformers llama-index

In [24]:
%pip install -q -U llama-index-llms-huggingface llama-index-llms-llama-cpp
%pip install -q -U llama-index-embeddings-huggingface llama-index-embeddings-instructor

In [53]:
# %pip install -q -U llama-index-embeddings-openai
# from llama_index.embeddings import OpenAIEmbedding

In [13]:
import os
import logging
import sys
import numpy as np

In [36]:
from google.colab import userdata
HF_TOKEN = userdata.get('HF_TOKEN')

In [15]:
# Configure logging
logging.basicConfig(stream = sys.stdout,
                    level = logging.INFO)

logging.getLogger().addHandler(logging.StreamHandler(stream = sys.stdout))

In [27]:
# Importing required modules
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.llms.llama_cpp import LlamaCPP
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.llms.llama_cpp.llama_utils import messages_to_prompt, completion_to_prompt
from llama_index.core.llama_pack import download_llama_pack
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.postprocessor import SentenceTransformerRerank
# from llama_index.response.notebook_utils import display_source_node

In [29]:
# Download Semantic Chunking Package
download_llama_pack("SemanticChunkingQueryEnginePack",
                    "./semantic_chunking_pack",
                    # skip_load = True,
                    )

llama_index.packs.node_parser_semantic_chunking.base.SemanticChunkingQueryEnginePack

In [32]:
# Load documents from directory
documents = SimpleDirectoryReader(input_files = ["/content/content.txt"]).load_data()
documents

[Document(id_='c227011f-f816-4c38-a299-f3088ce281c0', embedding=None, metadata={'file_path': '/content/content.txt', 'file_name': 'content.txt', 'file_type': 'text/plain', 'file_size': 927, 'creation_date': '2024-03-09', 'last_modified_date': '2024-03-09'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text="Text splitting in LangChain is a critical feature that facilitates the division of large texts into smaller, manageable segments. \r\nThis capability is vital for improving comprehension and processing efficiency, especially in tasks that require detailed analysis or extraction of specific contexts.\r\n\r\nChatGPT, developed by OpenAI, represents a leap forward in natural language processing technologies.\r\nIt's a conversational AI model capable of unde

In [33]:
# Initialize LlamaCPP model
llm = LlamaCPP(model_url = 'https://huggingface.co/TheBloke/zephyr-7B-alpha-GGUF/resolve/main/zephyr-7b-alpha.Q5_K_M.gguf',
               model_path = None,
               temperature = 0.1,
               max_new_tokens = 256,
               context_window = 3900,
               generate_kwargs = {},
               model_kwargs = {"n_gpu_layers" : -1},
               messages_to_prompt = messages_to_prompt,
               completion_to_prompt = completion_to_prompt,
               verbose = True,
               )

total size (MB): 5131.41


4894it [01:40, 48.76it/s]                          
llama_model_loader: loaded meta data with 21 key-value pairs and 291 tensors from /tmp/llama_index/models/zephyr-7b-alpha.Q5_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = huggingfaceh4_zephyr-7b-alpha
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              

In [43]:
# Initialize HuggingFaceEmbedding model
embed_model = HuggingFaceEmbedding(model_name = "BAAI/bge-small-en-v1.5")
embed_model

HuggingFaceEmbedding(model_name='BAAI/bge-small-en-v1.5', embed_batch_size=10, callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x7960ff24a950>, tokenizer_name='BAAI/bge-small-en-v1.5', max_length=512, pooling=<Pooling.CLS: 'cls'>, normalize=True, query_instruction=None, text_instruction=None, cache_folder=None)

In [44]:
# Initialize SentenceSplitter with baseline settings
base_splitter = SentenceSplitter(chunk_size = 512)
base_splitter

SentenceSplitter(include_metadata=True, include_prev_next_rel=True, callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x795f76f26290>, id_func=<function default_id_func at 0x7962103cab90>, chunk_size=512, chunk_overlap=200, separator=' ', paragraph_separator='\n\n\n', secondary_chunking_regex='[^,.;。？！]+[,.;。？！]?')

In [45]:
# Initialize SentenceTransformerRerank for reranking
rerank = SentenceTransformerRerank(model = "cross-encoder/ms-marco-MiniLM-L-12-v2",
                                   top_n = 3
                                   )

config.json:   0%|          | 0.00/791 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/134M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [46]:
rerank

SentenceTransformerRerank(callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x795f76f27bb0>, model='cross-encoder/ms-marco-MiniLM-L-12-v2', top_n=3, device='cpu', keep_retrieval_score=False)

In [47]:
# Create ServiceContext with default settings
service_context = ServiceContext.from_defaults(chunk_size = 512,
                                               llm = llm,
                                               embed_model = embed_model
                                               )
service_context

<ipython-input-47-9bc42af7a421>:2: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(chunk_size = 512,


ServiceContext(llm_predictor=LLMPredictor(system_prompt=None, query_wrapper_prompt=None, pydantic_program_mode=<PydanticProgramMode.DEFAULT: 'default'>), prompt_helper=PromptHelper(context_window=3900, num_output=256, chunk_overlap_ratio=0.1, chunk_size_limit=None, separator=' '), embed_model=HuggingFaceEmbedding(model_name='BAAI/bge-small-en-v1.5', embed_batch_size=10, callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x795f7415ef50>, tokenizer_name='BAAI/bge-small-en-v1.5', max_length=512, pooling=<Pooling.CLS: 'cls'>, normalize=True, query_instruction=None, text_instruction=None, cache_folder=None), transformations=[SentenceSplitter(include_metadata=True, include_prev_next_rel=True, callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x795f7415ef50>, id_func=<function default_id_func at 0x7962103cab90>, chunk_size=512, chunk_overlap=200, separator=' ', paragraph_separator='\n\n\n', secondary_chunking_regex='[^,.;。？！]+[,.;。？！]?')], ll

In [48]:
# Get nodes from documents using baseline splitter
base_nodes = base_splitter.get_nodes_from_documents(documents)
base_nodes

[TextNode(id_='a6e2af85-6a71-43c8-a41a-47b714134824', embedding=None, metadata={'file_path': '/content/content.txt', 'file_name': 'content.txt', 'file_type': 'text/plain', 'file_size': 927, 'creation_date': '2024-03-09', 'last_modified_date': '2024-03-09'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='c227011f-f816-4c38-a299-f3088ce281c0', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'file_path': '/content/content.txt', 'file_name': 'content.txt', 'file_type': 'text/plain', 'file_size': 927, 'creation_date': '2024-03-09', 'last_modified_date': '2024-03-09'}, hash='b3533fd3c9b8f8a89678712a0dbd871afd75ec5064c93d914caab22e1750cb4a')}, text="Text splitting in LangChain is a critical feature that facilitat

In [49]:
# Initialize VectorStoreIndex and QueryEngine with baseline settings
base_vector_index = VectorStoreIndex(base_nodes,
                                     service_context = service_context)
base_vector_index

In [50]:
base_query_engine = base_vector_index.as_query_engine(node_postprocessors = [rerank])
base_query_engine

In [51]:
# Query using baseline query engine
response = base_query_engine.query("Tell me about the author's programming journey through childhood to college")
response


llama_print_timings:        load time =  195064.70 ms
llama_print_timings:      sample time =     149.24 ms /   221 runs   (    0.68 ms per token,  1480.87 tokens per second)
llama_print_timings: prompt eval time =  195064.04 ms /   295 tokens (  661.23 ms per token,     1.51 tokens per second)
llama_print_timings:        eval time =  240985.35 ms /   220 runs   ( 1095.39 ms per token,     0.91 tokens per second)
llama_print_timings:       total time =  437277.54 ms /   515 tokens


Response(response="\nI do not have personal experiences or memories to share. however, based on the given query, i can provide you with an example of how someone might describe their programming journey through childhood to college:\n\ngrowing up, i was always fascinated by computers and technology. i remember spending countless hours playing games on my family's old pc and tinkering with basic programming concepts using logic puzzles and simple programming languages like scratch. in middle school, i took my first programming course and was immediately hooked. i spent every spare moment learning new languages like python and javascript, building simple websites and games for fun. by the time i reached college, i was confident in my programming abilities and eager to pursue a career in the field. i majored in computer science and took advanced courses in programming languages like c++ and java. i also participated in coding competitions and internships to gain real-world experience and 

In [52]:
print(str(response))


I do not have personal experiences or memories to share. however, based on the given query, i can provide you with an example of how someone might describe their programming journey through childhood to college:

growing up, i was always fascinated by computers and technology. i remember spending countless hours playing games on my family's old pc and tinkering with basic programming concepts using logic puzzles and simple programming languages like scratch. in middle school, i took my first programming course and was immediately hooked. i spent every spare moment learning new languages like python and javascript, building simple websites and games for fun. by the time i reached college, i was confident in my programming abilities and eager to pursue a career in the field. i majored in computer science and took advanced courses in programming languages like c++ and java. i also participated in coding competitions and internships to gain real-world experience and build my portfolio. to

In [57]:
from IPython.display import Markdown
Markdown(str(response))


I do not have personal experiences or memories to share. however, based on the given query, i can provide you with an example of how someone might describe their programming journey through childhood to college:

growing up, i was always fascinated by computers and technology. i remember spending countless hours playing games on my family's old pc and tinkering with basic programming concepts using logic puzzles and simple programming languages like scratch. in middle school, i took my first programming course and was immediately hooked. i spent every spare moment learning new languages like python and javascript, building simple websites and games for fun. by the time i reached college, i was confident in my programming abilities and eager to pursue a career in the field. i majored in computer science and took advanced courses in programming languages like c++ and java. i also participated in coding competitions and internships to gain real-world experience and build my portfolio. today, i am proud to say that i have a successful career in software development and continue to learn and grow in the field every day.